In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
import csv
from os.path import exists
from os import mkdir
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--start-maximized')
driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)

C:\Users\lynco\AppData\Local\Temp/ipykernel_10316/2263663545.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(),options=chrome_options)


In [3]:
driver.get("https://airbnb.com.br/")
sleep(3)
try:
    anuncio = driver.find_element(By.XPATH,'//*[@aria-label="Fechar"]')
    anuncio.click()
except:
    pass

In [4]:
caixa = driver.find_element(By.XPATH,'//div//*[contains(text(),"Qualquer lugar")]')
caixa.click()
sleep(2)

In [5]:
busca = driver.find_element(By.CSS_SELECTOR,'.ivycze5')
busca.send_keys('Itaboraí',Keys.RETURN)
sleep(2)

# busca = driver.find_element(By.XPATH,'//*[@id="/homes-1-input"]')


In [6]:
checkin = driver.find_element(By.XPATH,'//h2[contains(text(),"janeiro")]/ancestor::div[@class="_ytfarf"]//*[text()="20"]')
checkin.click()
sleep(2)
checkout = driver.find_element(By.XPATH,'//h2[contains(text(),"janeiro")]/ancestor::div[@class="_ytfarf"]//*[text()="23"]')
checkout.click()
sleep(2)
hospedes = driver.find_element(By.XPATH,'.//*[@id="search-tabpanel"]/div/div[5]/div[1]/div[1]/div/div[1]')
hospedes.click()
sleep(2)
adultos = driver.find_element(By.XPATH,'//*[@id="stepper-adults"]/button[2]')
adultos.click()
adultos.click()
sleep(2)
buscar = driver.find_element(By.CSS_SELECTOR,'._kaq6tx')
buscar.click()
sleep(5)

In [7]:
qtdPag = driver.find_element(By.XPATH,'//*[@id="site-content"]/div[2]/div[3]/div/div/nav/div/a[4]')
qtdPag = int(qtdPag.text)

In [ ]:
if(not exists('data/')):
    os.makdir('data/')


with open('data/airbnb.csv', 'w', encoding="utf-8-sig",newline='') as airbnb_file:
    campos = [
        'nome', 
        'local',
        'precoNoite',
        'total',
        'taxas',
        'avaliacoes',
        'ondeDormir', 
        'url'
    ]

    csv_writer = csv.DictWriter(airbnb_file, fieldnames=campos, delimiter=';', quoting=csv.QUOTE_ALL)
    csv_writer.writeheader()
    
    tempo = 3

    for i in range(qtdPag):
        print("-------------------",i+1,'-------------------\b')
        acomodacoes = driver.find_elements(By.CSS_SELECTOR,'.lwy0wad')
        for acomodacao in acomodacoes:
            acomodacao.click()
            sleep(tempo)
            driver.switch_to.window(driver.window_handles[1])
            
            try:
                nome = driver.find_element(By.CSS_SELECTOR,'._1n81at5')
                nome = nome.text
            except:
                nome = "---"
                
            try:
                local = driver.find_element(By.CSS_SELECTOR,'._9xiloll')
                local = local.text
            except:
                local = "---"

            try:
#                 precoNoite = driver.find_element(By.XPATH,'//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[1]')
                precoNoite = driver.find_element(By.XPATH,'//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div/div/span/span').text.split()[0][2:]
                precoNoite = int(precoNoite)
            except:
                precoNoite = "---"
                
            try:
                total = driver.find_element(By.XPATH,'//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/section/div[2]/div/span[2]')
                total = int(total.text[2:])
            except:
                total = '---'
            
            if(total != "---" and precoNoite != "---"):
                taxas = total - precoNoite*3
            else:
                taxas = '---'
            
            try:
                avaliacoes = driver.find_element(By.XPATH,'//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[2]/span/span[2]')
                avaliacoes = avaliacoes.text
            except:
                avaliacoes = "---"

            try:
                ondeDormir = driver.find_elements(By.CSS_SELECTOR,'._1a5glfg')
                ondeDormir = "".join(list(map(lambda x: x.text+',',ondeDormir)))
                if(ondeDormir == ""):
                    ondeDormir = driver.find_elements(By.CSS_SELECTOR,'._4zdnhq')
                    ondeDormir = "".join(list(map(lambda x: x.text+',',ondeDormir)))

            except:
                ondeDormir = "---"

            url = driver.current_url

            print(nome,local,precoNoite, total, taxas, avaliacoes,ondeDormir,url)
            
            csv_writer.writerow(dict(zip(campos, [nome, local, precoNoite, total, taxas, avaliacoes, ondeDormir, url])))


            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
            
        if(i != (qtdPag-1)):
            proximo = driver.find_element(By.XPATH,'//a[@aria-label="Próximo"]')
            proximo.click()
            sleep(tempo)
        else:
            print("Fim!")

------------------- 1 ------------------
Loft com Piscina e Academia no Centro de Itaboraí Centro, Rio de Janeiro, Brasil 135 540 135 4,77 · 1 cama de casal, 1 sofá-cama, 1 colchão extra, https://www.airbnb.com.br/rooms/48469638?adults=2&check_in=2023-01-20&check_out=2023-01-23&source_impression_id=p3_1673760188_ntvSp%2FmpEx%2FgUhvf
Flat mobiliado em Itaboraí Itaboraí, Rio de Janeiro, Brasil 180 619 79 --- 2 camas de solteiro,1 sofá, https://www.airbnb.com.br/rooms/14087357?adults=2&check_in=2023-01-20&check_out=2023-01-23&source_impression_id=p3_1673760191_lY9Kouhs6vSAEF1c
Solar do Artista Quarto 2 Itaboraí, Rio de Janeiro, Brasil 116 399 51 --- 1 cama de casal, https://www.airbnb.com.br/rooms/795480651209051583?adults=2&check_in=2023-01-20&check_out=2023-01-23&source_impression_id=p3_1673760195_9rQt4W3xPeTYS1Wn
Oásis em Itaboraí: Flat no Enterprise Suíte Itaboraí, Rio de Janeiro, Brasil 150 515 65 ---  https://www.airbnb.com.br/rooms/735517556299828281?adults=2&check_in=2023-01-20&ch

In [ ]:

local.text

In [ ]:
driver.switch_to.window(driver.window_handles[1])
driver.title